# Lab 1: image rectification

This lab has two main goals:

1) Get more familiar with the hierarchy of 2D transformation <br>

2) Remove the projective distortion of an image of a planar object (image rectification)

The following file combines some text cells (Markdown cells) and code cells. Some parts of the code need to be completed. All tasks you need to complete are marked in <span style='color:Green'> green.  </span>

In [ ]:
from PIL import Image, ImageDraw
import numpy as np
from scipy.ndimage import map_coordinates
from numpy import linalg as LA
from math import ceil
from utils import line_draw, plot_img

## **1. Applying image transformations**

In this first part of the lab you will apply different types of 2D transformations to a given image. For that, you first need to create a function that applies a homography to animage.

<span style='color:Green'> - Create the function  *apply_H* that gets as input a homography and
an image and returns the image transformed by the homography. </span>

Note: The size of the transformed image has to be automatically set so as to 
contain the whole transformed image.
You will need to interpolate the image values at some points, for that,
you may use the function *scipy.ndimage.map_coordinates*

In [ ]:
from typing import Tuple
def warp(H:np.array,p:Tuple):
    """
    Warp a point based on a given Homography matrix
    
    Returns:
        Tuple of points
    """
    x1,x2 = p
    x1, x2, x3 = H@np.array([x1,x2,1])
    if x3 != 0:
        return x1/x3, x2/x3
    return x1, x2

def apply_H(I:np.array, H:np.array):
    """
    Apply homography matrix to a given image
    
    Returns:
        transformed image
    """
    h,w,c = I.shape
    tl = warp(H,(0,0)) # top left
    tr = warp(H,(w,0)) # top right
    bl = warp(H,(0,h)) # bottom left
    br = warp(H,(w,h)) # bottom right
    corners = np.array([tl,tr,bl,br])

    #get min and max coordinates in the new space
    min_x = np.ceil(corners.min(axis=0)[0])
    max_x = np.floor(corners.max(axis=0)[0])
    min_y = np.ceil(corners.min(axis=0)[1])
    max_y = np.floor(corners.max(axis=0)[1])

    # get size of canvas
    width_canvas, height_canvas = max_x-min_x,max_y-min_y

    # generate a grid corresponding to new space coordinates
    xx,yy = np.meshgrid(np.arange(min_x,max_x),np.arange(min_y,max_y))

    # make a matrix with all new points in homogenous coordinates
    dstpointsH = np.array([xx.flatten(),yy.flatten(),np.ones_like(xx.flatten())])
    H_inv = np.linalg.inv(H)
    # map them back to their source points
    # src_points = H_inv@dstpointsH
    src_points = np.matmul(H_inv,dstpointsH)

    # get src_x and src_y in meshgrid-like coordinates
    src_x = np.reshape(src_points[0]/src_points[2], xx.shape)
    src_y = np.reshape(src_points[1]/src_points[2], yy.shape)

    #draw canvas
    out = np.zeros((int(height_canvas),int(width_canvas),3))

    # using map_coordinates in by passing in a meshgrid is one of the best parts about this
    # function. You must be familiar with its input arguments to make the best use of it.
    for i in range(c):
        out[:,:,i] = map_coordinates(I[:,:,i],[src_y,src_x])
    return np.uint8(out)

### **1.1 Similarities**

<span style='color:Green'> - Complete the code below by generating a matrix H which produces a similarity transformation. </span>

In [ ]:
#H = ... # complete ...

img_path = "./Data/0005_s.png"
I = Image.open(img_path)
I_sim = apply_H(np.array(I), H)

plot_img(I)
plot_img(I_sim)

### **1.2 Affinities**

<span style='color:Green'> - Complete the code below by generating a matrix H which produces an affine transformation.  </span>

<span style='color:Green'> - Decompose the affinity in four transformations: two
rotations, a scale, and a translation (you may use function *numpy.linalg.svd* for that).  </span>

<span style='color:Green'> - Verify that the product of the four previous transformations
produces the same matrix H as above.  </span>

<span style='color:Green'> - verify that the proper sequence of the four previous
transformations over the image I produces the same transformed image as before.  </span>

In [ ]:
#H = ... # complete

I_aff = apply_H(np.array(I), H)

plot_img(I)
plot_img(I_aff)

### **1.3 Projective transformations (Homographies)**

<span style='color:Green'> - Complete the code below by generating a matrix H which produces a projective transformation.  </span>

In [ ]:
#H = ... # complete

I_proj = apply_H(np.array(I), H)

plot_img(I)
plot_img(I_proj)

## **2. Affine Rectification**

This step is needed in order to rectify an image in a stratified way, where we first perform affine rectification (current section) and then metric rectification (Section 3).

In [ ]:
# load images and lines
img_path = "./Data/0000_s.png"
I = Image.open(img_path)

lines_path = "./Data/0000_s_info_lines.txt"
A = np.loadtxt(lines_path)

# points of interest
i = 423 # line index
p1 = [A[i, 0], A[i, 1], 1] # initial point in line i
p2 = [A[i, 2], A[i, 3], 1] # final point in line i
i = 239
p3 = [A[i, 0], A[i, 1], 1]
p4 = [A[i, 2], A[i, 3], 1]
i = 711
p5 = [A[i, 0], A[i, 1], 1]
p6 = [A[i, 2], A[i, 3], 1]
i = 564
p7 = [A[i, 0], A[i, 1], 1]
p8 = [A[i, 2], A[i, 3], 1]

<span style='color:Green'> - Compute the lines l1, l2, l3, l4, that pass through the different pairs of points.  </span>

In [ ]:
l1, l2, l3, l4 = # complete ...

In [ ]:
# show the chosen lines in the image
canv = ImageDraw.Draw(I)
point_color = (0, 0, 255)
line_draw(l1, canv, I.size)
line_draw(l2, canv, I.size)
line_draw(l3, canv, I.size)
line_draw(l4, canv, I.size)

# The displayed lines will alter image I so we have to reopen the original image after the plot
plot_img(I)
I = Image.open(img_path)

<span style='color:Green'> - Compute the homography that affinely rectifies the image </span>
ToDo: 

<span style='color:Green'> - Compute the transformed lines lr1, lr2, lr3, lr4 and
      show the transformed lines in the transformed image. </span>
      
<span style='color:Green'> - To evaluate the results, compute the angle between the different pair 
      of lines before and after the image transformation. </span>
ToDo: 
**better rephrase more precisely (see two options and also expression of line slope)**    

## **3. Metric Rectification**

### **3.1 Metric rectification after the affine rectification (stratified solution)**

<span style='color:Green'> - Write the code that performs the metric rectification (after the affine rectification). </span>

As qualitative evaluation method you can display the images (before and after the metric rectification) with the chosen lines printed on it.
      
<span style='color:Green'> - Compute the angles between the pair of lines before and after rectification. Comment the result. </span>
      


## **4. Affine and Metric Rectification of the left facade of image 0001**

<span style='color:Green'> - Write the code that rectifies the left facade of image 0001 with
      the stratified method.  </span>
      
Note: For a better visualization of the result crop the initial image so that only the left facade is visible.

<span style='color:Green'> - Show the (properly) transformed lines that are used in every step.  </span>
      
      

## **5. OPTIONAL: Metric Rectification in a single step**

<span style='color:Green'> - Write the code that performs metric rectification in a single step (algorithm pages 55-57, Hartley-Zisserman book). </span>

Note: Use 5 pairs of orthogonal lines. You may consider that windows are square.